In [18]:
import time
from collections import OrderedDict

In [14]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [3]:
data_dir = '/home/jupyter/data/Cat_Dog_data'

In [4]:
imagenet_mean, imagenet_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

In [5]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(imagenet_mean, imagenet_std)])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [6]:
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(imagenet_mean, imagenet_std)])

test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [8]:
model = models.densenet121(pretrained=True)

In [10]:
model.classifier

Linear(in_features=1024, out_features=1000, bias=True)

In [13]:
for param in model.parameters():
    param.requires_grad = False

In [15]:
classifier = nn.Sequential(OrderedDict([
                  ('fc1', nn.Linear(1024, 500)),
                  ('relu', nn.ReLU()),
                  ('fc2', nn.Linear(500, 2)),
                  ('output', nn.LogSoftmax(dim=1))]))

In [16]:
model.classifier = classifier

In [17]:
model.classifier

Sequential(
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (output): LogSoftmax()
)

In [20]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [53]:
model = models.densenet121(pretrained=True)

In [54]:
for param in model.parameters():
    param.requires_grad = False

In [55]:
model.classifier

Linear(in_features=1024, out_features=1000, bias=True)

In [56]:
classifier = nn.Sequential(nn.Linear(1024, 256),
                           nn.ReLU(),
                           nn.Dropout(0.2),
                           nn.Linear(256, 2),
                           nn.LogSoftmax(dim=1))

In [57]:
model.classifier = classifier

In [58]:
model.classifier

Sequential(
  (0): Linear(in_features=1024, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2)
  (3): Linear(in_features=256, out_features=2, bias=True)
  (4): LogSoftmax()
)

In [59]:
criterion = nn.NLLLoss()

In [60]:
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [61]:
model.to(device);

In [62]:
epochs = 2
steps = 0
running_loss = 0
print_every = 5

In [ ]:
for epoch in range(epochs):
    for images, labels in trainloader:
        steps += 1
        
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        log_probs = model(images)
        loss = criterion(log_probs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            test_loss = 0
            accuracy = 0
            
            with torch.no_grad():
                for images, labels in trainloader:
                    images = images.to(device)
                    labels = labels.to(device)
                    
                    log_probs = model(images)
                    loss = criterion(log_probs, labels)
                    
                    test_loss += loss.item()
                    
                    # calculate accuracy
                    probs = torch.exp(log_probs)
                    top_prob, top_class = probs.topk(1, dim=1)
                    
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs} "
                  f"Train loss: {running_loss/print_every:.3f} "
                  f"Test loss: {test_loss/len(testloader):.3f} "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            
            running_loss = 0
            model.train()     

Epoch 1/2 Train loss: 0.804 Test loss: 8.514 Test accuracy: 4.498
Epoch 1/2 Train loss: 0.532 Test loss: 3.987 Test accuracy: 6.815
Epoch 1/2 Train loss: 0.427 Test loss: 2.450 Test accuracy: 7.651
Epoch 1/2 Train loss: 0.265 Test loss: 1.930 Test accuracy: 7.852
Epoch 1/2 Train loss: 0.292 Test loss: 1.822 Test accuracy: 7.984
Epoch 1/2 Train loss: 0.177 Test loss: 1.355 Test accuracy: 8.235
Epoch 1/2 Train loss: 0.216 Test loss: 1.914 Test accuracy: 7.878
Epoch 1/2 Train loss: 0.305 Test loss: 1.715 Test accuracy: 8.029
Epoch 1/2 Train loss: 0.174 Test loss: 1.557 Test accuracy: 8.070
Epoch 1/2 Train loss: 0.266 Test loss: 1.505 Test accuracy: 8.095
Epoch 1/2 Train loss: 0.184 Test loss: 2.539 Test accuracy: 7.695
Epoch 1/2 Train loss: 0.242 Test loss: 1.526 Test accuracy: 8.095
Epoch 1/2 Train loss: 0.219 Test loss: 1.411 Test accuracy: 8.168
Epoch 1/2 Train loss: 0.148 Test loss: 1.583 Test accuracy: 8.106
Epoch 1/2 Train loss: 0.135 Test loss: 1.269 Test accuracy: 8.232
Epoch 1/2 

In [ ]:
with torch.no_grad():
    for inputs, labels in testloader:

        # Calculate accuracy
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()



In [ ]:
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()